In [ ]:
## This file is part of Jax Geometry
#
# Copyright (C) 2021, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/jaxgeometry
#
# Jax Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Jax Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Jax Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Horizontal/vertical splittings and lifted sub-elliptic diffusions

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
## SO(3)
from src.groups.SON import *
G = SON(3)
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
from src.group import invariant_metric
invariant_metric.initialize(G)

from src.group import energy
energy.initialize(G)

from src.group.quotient import *

In [ ]:
q = jnp.array([1e-3,0.,0.])
g = G.psi(q)
v = jnp.array([0.,1.,1.])
p = G.sharppsi(q,v)

# base point on S2
x = M.coords(jnp.array([0.,0.]))
# projection
proj = lambda g: M.act(g,M.F(x))

In [ ]:
# sample Brownian paths on SO(3) and S^2. Note bi-invariant metric let's process descend to Brownian motion on S^2
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

sigma = jnp.eye(G.dim) # diffusivity / metric
K = 1
_dts = dts(n_steps=1000)
gss = np.zeros((K,_dts.shape[0])+g.shape)
for i in range(K):
    (ts,gs,_) = G.Brownian_inv(g,_dts,dWs(G.dim,_dts),sigma)
    gss[i] = gs

# plot samples
newfig()
for i in range(K):
    G.plot_path(gs,linewidth=0.1,alpha=0.1)
plt.show()

# on S2
gss_M = np.zeros((K,_dts.shape[0],M.emb_dim,))
newfig()
M.plot()
for i in range(K):
    gss_M[i] = M.acts(gss[i],M.F(x))
    M.plot_path(gss_M[i])
plt.show()

In [ ]:
# truncated SO(3) Brownian motion
sde_Brownian_inv_horz = get_sde_horz(G.sde_Brownian_inv,proj,G,M)
sde_Brownian_inv_fiber = get_sde_fiber(G.sde_Brownian_inv,proj,G,M)
Brownian_inv_horz = lambda g,dts,dWt,sigma=jnp.eye(G.dim): integrate_sde(sde_Brownian_inv_horz,
                                                  integrator_stratonovich,
                                                  None,g,None,dts,dWt,sigma)[0:3]
Brownian_inv_fiber = lambda g,dts,dWt,sigma=jnp.eye(G.dim): integrate_sde(sde_Brownian_inv_fiber,
                                                  integrator_stratonovich,
                                                  None,g,None,dts,dWt,sigma)[0:3]


# make simulations, horizontal
K = 1
_dts = dts(n_steps=1000)
gss = np.zeros((K,_dts.shape[0])+g.shape)
for i in range(K):
    (ts,gs,_) = Brownian_inv_horz(g,_dts,dWs(G.dim,_dts),sigma)
    gss[i] = gs

# plot samples
newfig()
for i in range(K):
    G.plot_path(gs,linewidth=0.1,alpha=0.1)
plt.show()

# on S2
gss_M = np.zeros((K,_dts.shape[0],M.emb_dim,))
newfig()
M.plot()
for i in range(K):
    gss_M[i] = M.acts(gss[i],M.F(x))
    M.plot_path(gss_M[i])
plt.show()

# make simulations, vertical
gss = np.zeros((K,_dts.shape[0])+g.shape)
for i in range(K):
    (ts,gs,_) = Brownian_inv_fiber(g,_dts,dWs(G.dim,_dts),sigma)
    gss[i] = gs

# plot samples
newfig()
for i in range(K):
    G.plot_path(gs,linewidth=0.1,alpha=0.1)
plt.show()

# on S2
gss_M = np.zeros((K,_dts.shape[0],M.emb_dim,))
newfig()
M.plot()
for i in range(K):
    gss_M[i] = M.acts(gss[i],M.F(x))
    M.plot_path(gss_M[i])
plt.show()

In [ ]:
# lift from S^2 to SO(3)

# S^2 Brownian motion
from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

sde_Brownian_inv_lifted = get_sde_lifted(M.sde_Brownian_coords,proj,G,M)
Brownian_inv_lifted = lambda g,dts,dWt,sigma=jnp.eye(G.dim): integrate_sde(sde_Brownian_inv_lifted,
                                                  integrator_stratonovich,
                                                  None,g,x[1],dts,dWt,sigma,1.)[0:3]

# make simulations
K = 1
_dts = dts(n_steps=1000)
gss = np.zeros((K,_dts.shape[0])+g.shape)
for i in range(K):
    (ts,gs,_) = Brownian_inv_lifted(g,_dts,dWs(M.dim,_dts),sigma)
    gss[i] = gs

# plot samples
newfig()
for i in range(K):
    G.plot_path(gs,linewidth=0.1,alpha=0.1)
plt.show()

# on S2
gss_M = np.zeros((K,_dts.shape[0],M.emb_dim,))
newfig()
M.plot()
for i in range(K):
    gss_M[i] = M.acts(gss[i],M.F(x))
    M.plot_path(gss_M[i])
plt.show()